In [2]:
from pyspark.sql import Row

In [3]:
update=False

In [4]:
sqlContext.sql('use weizmann')
sqlContext.sql('show databases').show()

+--------+
|  result|
+--------+
| aibgraz|
| default|
|    test|
|   test2|
|weizmann|
+--------+



# create database

In [1]:
image_dataset='image_dataset'

In [7]:
path_prefix_ground='/home/sh/Data/TestData/WeizMaan_horse/weizmann_horse_db/figure_ground'
path_prefix_gray ='/home/sh/Data/TestData/WeizMaan_horse/weizmann_horse_db/gray'
path_prefix_rgb ='/home/sh/Data/TestData/WeizMaan_horse/weizmann_horse_db/rgb'

if update==True:
    sqlContext.sql('drop table if exists %s'%(image_dataset))
    print('[info] info mode, drop table if exists')

if image_dataset not in sqlContext.sql('show tables').rdd.map(lambda row : row.tableName).collect():
    import os
    import os.path
    list_dataset =[]
    list_file = os.listdir(path_prefix_rgb)
    id = 1
    for file_name in list_file:
        path_rgb = os.path.join(path_prefix_rgb, file_name)
        path_gray = os.path.join(path_prefix_gray, file_name)
        path_ground = os.path.join(path_prefix_ground, file_name)
    
        list_dataset.append(Row(id=id, rgb=path_rgb, gray=path_gray, ground = path_ground))
        id = id + 1  
    
    rdd_dataset = sc.parallelize(range(0, len(list_dataset))).map(lambda i: list_dataset[i])
    df_dataset = sqlContext.createDataFrame(rdd_dataset)
    df_dataset.write.saveAsTable(name=image_dataset)
    print('[info] Done')
else:
    print('[info] table exists')

[info] table exists


In [8]:
sqlContext.sql('show tables').show()

+-------------+-----------+
|    tableName|isTemporary|
+-------------+-----------+
|image_dataset|      false|
+-------------+-----------+



# dataset selection

In [9]:
image_split = 'image_split'

In [11]:
def split(row):
    if row.id <=200:
        split='train'
    else:
        split='test'
    return Row(id=row.id, split=split)

In [13]:
rdd_dataset=sqlContext.sql('SELECT id FROM %s'%(image_dataset)).rdd
rdd_split=rdd_dataset.map(split)
df_split=sqlContext.createDataFrame(rdd_split)
df_split.write.saveAsTable(name=image_split)

# mathmatics background

## CRF-based segmentation
Given an Image $\mathbf{x}$ and its corresponding labelling $\mathbf{y}$, CRF(conditional random filed) models the conditional distribution of the form
$$
P(\mathbf{y} | \mathbf{x}; \mathbf{W})=\frac{1}{Z}exp(-E(\mathbf{y,x;W}))
$$
where $\mathbf{w}$ are paremeters and Z is the normalization term. The energy E of an image $\mathbf{x}$ with segmentation labels $\mathbf{y}$ over the the nodes $\mathcal{N}$ and edges $\mathcal{S}$ takes the following form:
$$
E(\mathbf{y,x;w})=\sum_{p\in\mathcal{N}}{\Phi^{(1)}(y^p, \mathbf{x}; \mathbf{w})} + \sum_{(p,q)\in\mathcal{S}}\Phi^{(2)}(y^p, y^q, \mathbf{x}; \mathbf{w}).
$$

Here, $\mathbf{x}\in\mathcal{X}$; $\mathbf{y}\in\mathcal{y}$ and $\Phi^{(1)}$,$\Phi^{(2)}$ are the unary and pairwise potentials.
There are two setps in CRF-based segment, (1) learning the model parameters($w$) from the trainning data; (2) inferring a most likely label for the test data given the learned parameters. The segmentation problem thus reduced to find label $\mathbf{y}$ with which get minimum energy, thus :
$$
\mathbf{y^*} = \arg\min_{\mathbf{y} \in \mathcal{y} }{ E(\mathbf{y, x; w})}.
$$

## parameter of CRF
To learn CRF with large margin framework, we make energy linear in the parameter $\mathbf{w}$, so we define unary and pairwise potentials as:
$$
\Phi^{(1)}(y^p, \mathbf{x}; \mathbf{w})=<\mathbf{w}^{(1)},  \phi^{(1)}(y^p, \mathbf{x})>
\\
\Phi^{(2)}(y^p, y^q, \mathbf{x}; \mathbf{w}) = <\mathbf{w}^{(2)},  \phi^{(2)}(y^p, y^q, \mathbf{x})>
$$
where $\phi^{(1)}$ and $\phi^{(2)}$ are the unary and pairwise feature of superpixel $\mathbf{x}$. So we get:
$$
\mathbf{w}=[\mathbf{w}^{(1)}, \mathbf{w}^{(2)}]
$$
where $[\mathbf{w}^{(1)}, \mathbf{w}^{(2)}]$ means $\mathbf{w}^{(1)}$ concatenate $\mathbf{w}^{(2)}$.

# learning the model parameters

## construct CRF from superpixel
Superpixel become standard preprocess of 
## maximum 

In [19]:
sql_cmd='SELECT %s.id, %s.rgb, %s.ground \
         FROM %s join %s \
         ON %s.id = %s.id \
         WHERE %s.split=="train" '\
         %(image_dataset, image_dataset,image_dataset,\
           image_split, image_dataset,\
           image_split, image_dataset,\
           image_split)

rdd_image_train=sqlContext.sql(sql_cmd).rdd
print(rdd_image_train.count())

200
